# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. AGHAV SAYALI SAKHARAM (2023ac05435)
2. DULAL DAS (2023ac05041)
3. JAWAHARLAL RAJAN S (2023ac05504)
4. LAKSHMISRINIVAS PERAKAM (2023ac05540)
5. SUBHRANSU MISHRA (2023ac05489)

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

In [146]:
#Coding begins here
import heapq
import math
from collections import deque
import time

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [147]:
#Code Block : Set Initial State (Must handle dynamic inputs)
# Assume O - office, H - home, X - road blocks, B - other buildings
grid = [
    ['O', ' ', 'X', 'X', ' ', ' '],
    ['B', ' ', 'X', ' ', ' ', ' '],
    [' ', ' ', ' ', ' ', 'X', ' '],
    ['B', ' ', 'X', ' ', ' ', 'B'],
    [' ', ' ', ' ', ' ', 'B', ' '],
    [' ', 'B', ' ', 'X', ' ', 'H']
]

for row in grid:
    print(row)

def set_initial_state():
    return grid

['O', ' ', 'X', 'X', ' ', ' ']
['B', ' ', 'X', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', 'X', ' ']
['B', ' ', 'X', ' ', ' ', 'B']
[' ', ' ', ' ', ' ', 'B', ' ']
[' ', 'B', ' ', 'X', ' ', 'H']


In [148]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

# UCS
def calculate_cost_ucs(number_of_square, points):
    # cost is lower the better
    # higher points is better
    return number_of_square - points

#RBFS
def calculate_heuristic_rbfs(current_position, goal_position):
    """
    Calculate the Euclidean distance heuristic between the current position and the goal.
    """
    return math.sqrt((current_position[0] - goal_position[0])**2 + (current_position[1] - goal_position[1])**2)


def calculate_penalty_rbfs(city_grid, position):
    """
    Calculate the penalty based on proximity to buildings ('B') and roadblocks ('X').
    """
    x, y = position
    penalty = 0

    # Adjacent positions to check for penalties
    adjacent_offsets = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    for dx, dy in adjacent_offsets:
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(city_grid) and 0 <= ny < len(city_grid[0]):
            if city_grid[nx][ny] == 'B':
                penalty += 5  # Add reward for proximity to a building
            elif city_grid[nx][ny] == 'X':
                penalty -= 3  # impose penalty for proximity to a roadblock

    return penalty

In [149]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented


# Directions for moving in the grid (right, left, down, up, and diagonals)
straight_directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
diagonal_directions = [(1, 1), (1, -1), (-1, 1), (-1, -1)]

def get_neighbors_rbfs(city_grid, current_position, current_cost, goal_position, visited_nodes):
    """
    Generate and return valid neighboring positions for the GPS agent.
    Straight movements are prioritized, and diagonal movements are only considered when necessary.
    """
    neighbors = []

    # Explore straight directions first
    for direction in straight_directions:
        new_position = (current_position[0] + direction[0], current_position[1] + direction[1])
        if 0 <= new_position[0] < len(city_grid) and 0 <= new_position[1] < len(city_grid[0]):
            if city_grid[new_position[0]][new_position[1]] != 'B' and city_grid[new_position[0]][new_position[1]] != 'X':
                if new_position not in visited_nodes:
                    penalty = calculate_penalty_rbfs(city_grid, new_position)
                    movement_cost = current_cost + 1 + penalty  # Straight movement cost with penalty
                    heuristic_cost = calculate_heuristic_rbfs(new_position, goal_position)
                    neighbors.append((new_position, movement_cost, heuristic_cost))

    # If no unvisited straight paths are available, explore diagonal directions
    if not neighbors:
        for direction in diagonal_directions:
            new_position = (current_position[0] + direction[0], current_position[1] + direction[1])
            if 0 <= new_position[0] < len(city_grid) and 0 <= new_position[1] < len(city_grid[0]):
                if city_grid[new_position[0]][new_position[1]] != 'B' and city_grid[new_position[0]][new_position[1]] != 'X':
                    if new_position not in visited_nodes:
                        penalty = calculate_penalty_rbfs(city_grid, new_position)
                        movement_cost = current_cost + 1 - 3 + penalty  # Diagonal movement cost (3) with penalty
                        heuristic_cost = calculate_heuristic_rbfs(new_position, goal_position)
                        neighbors.append((new_position, movement_cost, heuristic_cost))

    return neighbors


In [150]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

def goal_test_rbfs(current_position, goal_position):
    """
    Check if the current position matches the goal position.
    """
    return current_position == goal_position


def goal_test_ucs(x, y):
    return grid[x][y] == 'H'

### 2.	Definition of Algorithm 1 (Recursive Best-First Search (RBFS))

In [151]:
#Code Block : Function for algorithm 1 implementation

# RBFS Algorithm Implementation

def gps_navigation_agent_rbfs(city_grid, start_position, goal_position):
    """
    Implement Recursive Best-First Search (RBFS) algorithm to navigate the city grid.
    The agent aims to find the safest and shortest path from start to goal.
    """
    # Initialize the queue with the starting position
    navigation_queue = deque([(start_position, 0, calculate_heuristic_rbfs(start_position, goal_position), float('inf'))])
    nodes_expanded = 0
    path_tracker = {start_position: None}
    path_costs = {start_position: 0}
    visited_nodes = set()

    while navigation_queue:
        current_position, current_cost, heuristic_cost, f_limit = navigation_queue.pop()
        nodes_expanded += 1
        visited_nodes.add(current_position)

        # Verbose output for current node expansion and path cost
        print(f"Expanding Node: {current_position} with Path Cost: {current_cost}")

        # Check if the goal has been reached
        if goal_test_rbfs(current_position, goal_position):
            # Backtrack to generate the final path
            final_path = []
            while current_position:
                final_path.append(current_position)
                current_position = path_tracker[current_position]
            final_path.reverse()
            return final_path, path_costs[goal_position], nodes_expanded

        # Generate neighbors for the current position
        neighbors = get_neighbors_rbfs(city_grid, current_position, current_cost, goal_position, visited_nodes)
        if not neighbors:
            continue

        # Sort neighbors by the total estimated cost (g + h)
        neighbors = sorted(neighbors, key=lambda x: x[1] + x[2])

        alternative_f_limit = float('inf')

        # Explore each neighbor
        for i in range(len(neighbors)):
            neighbor_position, neighbor_cost, neighbor_heuristic = neighbors[i]
            if neighbor_position not in visited_nodes and (neighbor_position not in path_costs or neighbor_cost < path_costs[neighbor_position]):
                path_tracker[neighbor_position] = current_position
                path_costs[neighbor_position] = neighbor_cost

                if neighbor_cost + neighbor_heuristic <= f_limit:
                    navigation_queue.append((neighbor_position, neighbor_cost, neighbor_heuristic, f_limit))
                else:
                    alternative_f_limit = min(alternative_f_limit, neighbor_cost + neighbor_heuristic)

        if navigation_queue and navigation_queue[-1][2] + navigation_queue[-1][1] > f_limit:
            f_limit = alternative_f_limit

    return None, None, nodes_expanded  # No path found


### 3.	Definition of Algorithm 2 (Uniform Cost Search (UCS))

In [152]:
#Code Block : Function for algorithm 2 implementation

# UCS
# Dimensions of the grid
rows = len(grid)
cols = len(grid[0])

def ucs(start):
    """
    Implement Uniform Cost Search (UCS) algorithm to navigate the city grid.
    The agent aims to find the safest and shortest path from start to goal.
    """
    # Priority queue for UCS
    queue = []
    heapq.heappush(queue, (calculate_cost_ucs(0, 0), 0, 0, start))  # (priority, cost, points, position)

    # Set for visited nodes
    visited = set()

    while queue:
        priority, cost, points, (x, y) = heapq.heappop(queue)

        # If the goal is reached
        if goal_test_ucs(x, y):
            print(f"visiting ({x},{y}), cost {cost}, points {points}, Goal Test: True")
            return cost, points, visited

        if (x, y) in visited:
            continue

        visited.add((x, y))
        print(f"visiting ({x},{y}), cost {cost}, points {points}, Goal Test: False")
        # Flag to check if any straight move is possible
        straight_move_possible = False

        for dx, dy in straight_directions:
            nx, ny = x + dx, y + dy

            if 0 <= nx < rows and 0 <= ny < cols and (nx, ny) not in visited and grid[nx][ny] not in ('X', 'B'):
                straight_move_possible = True
                new_cost = cost + 1
                new_points = points

                # Check for adjacent 'X' and 'B'
                for ddx, ddy in straight_directions:
                    ax, ay = nx + ddx, ny + ddy
                    if 0 <= ax < rows and 0 <= ay < cols:
                        if grid[ax][ay] == 'X':
                            new_points -= 3
                        elif grid[ax][ay] == 'B':
                            new_points += 5

                heapq.heappush(queue, (calculate_cost_ucs(new_cost, new_points), new_cost, new_points, (nx, ny)))

        # If no straight path was possible, allow diagonal movements
        if not straight_move_possible:
            #penalty of 3
            diagonal_points = points - 3
            for dx, dy in diagonal_directions:
                nx, ny = x + dx, y + dy

                if 0 <= nx < rows and 0 <= ny < cols and (nx, ny) not in visited and grid[nx][ny] not in ('X', 'B'):
                    new_cost = cost + 1
                    new_points = diagonal_points

                    # Check for adjacent 'X' and 'B'
                    for ddx, ddy in straight_directions + diagonal_directions:
                        ax, ay = nx + ddx, ny + ddy
                        if 0 <= ax < rows and 0 <= ay < cols:
                            if grid[ax][ay] == 'X':
                                new_points -= 3
                            elif grid[ax][ay] == 'B':
                                new_points += 5

                    heapq.heappush(queue, (calculate_cost_ucs(new_cost, new_points), new_cost, new_points, (nx, ny)))

    return float('inf'), 0, visited


### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [153]:
#Code Block : Function & call to get inputs (start/end state)

def get_dynamic_inputs():
    """
    Provide dynamic start and goal positions.
    """
    start_position = (0, 0)
    goal_position = (5, 5)
    return start_position, goal_position

### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [154]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

def run_gps_navigation_agent_rbfs():
    # Example grid
    city_grid = set_initial_state()

    # Get static start and goal positions
    start_position, goal_position = get_dynamic_inputs()

    # Measure time complexity
    start_time = time.time()

    # Run the GPS Navigation Agent
    optimal_path, total_path_cost, total_nodes_expanded = gps_navigation_agent_rbfs(city_grid, start_position, goal_position)

    # Measure time complexity
    end_time = time.time()
    time_complexity = end_time - start_time

    min_cost_rbfs = len(optimal_path)

    # Verbose output of the results
    if optimal_path:
        print("Optimal Path Found:", optimal_path)
        print("Optimal Path Len", len(optimal_path))
        print("Total Path Cost:", total_path_cost)
    else:
        print("No path found")
    print("Space Complexity: Number of nodes expanded =", total_nodes_expanded)
    print("Time Complexity: Time taken =", time_complexity, "seconds")
    return total_nodes_expanded, time_complexity,min_cost_rbfs,total_path_cost
space_complexity_rbfs, time_complexity_rbfs,min_cost_rbfs,max_points_rbfs = run_gps_navigation_agent_rbfs()

Expanding Node: (0, 0) with Path Cost: 0
Expanding Node: (0, 1) with Path Cost: -2
Expanding Node: (1, 1) with Path Cost: 1
Expanding Node: (2, 1) with Path Cost: 2
Expanding Node: (2, 0) with Path Cost: 13
Expanding Node: (3, 1) with Path Cost: 5
Expanding Node: (4, 1) with Path Cost: 11
Expanding Node: (4, 0) with Path Cost: 17
Expanding Node: (5, 0) with Path Cost: 23
Expanding Node: (4, 2) with Path Cost: 9
Expanding Node: (5, 2) with Path Cost: 12
Expanding Node: (4, 3) with Path Cost: 12
Expanding Node: (3, 3) with Path Cost: 10
Expanding Node: (3, 4) with Path Cost: 18
Expanding Node: (4, 5) with Path Cost: 26
Expanding Node: (5, 5) with Path Cost: 27
Optimal Path Found: [(0, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (4, 2), (4, 3), (3, 3), (3, 4), (4, 5), (5, 5)]
Optimal Path Len 12
Total Path Cost: 27
Space Complexity: Number of nodes expanded = 16
Time Complexity: Time taken = 0.00019311904907226562 seconds


In [155]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))

def run_gps_navigation_agent_ucs():
    start_position, goal_position = get_dynamic_inputs()
    print(f"Starting UCS from {start_position}")
    return ucs(start_position)

import tracemalloc
def measure_complexity(func, *args):
    tracemalloc.start()
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    time_taken = end_time - start_time
    space_taken = peak / 10**6  # Convert to MB
    return result, time_taken, space_taken

(min_cost_ucs, max_points_ucs, visited_nodes), time_complexity_ucs, space_taken_ucs = measure_complexity(run_gps_navigation_agent_ucs)
space_complexity_ucs = len(visited_nodes)
print(f"Minimum cost: {min_cost_ucs}, Maximum points: {max_points_ucs}, Time Complexity: {time_complexity_ucs}, Space Complexity: {space_complexity_ucs}, Actual Memory used: {space_taken_ucs}")


Starting UCS from (0, 0)
visiting (0,0), cost 0, points 0, Goal Test: False
visiting (0,1), cost 1, points -3, Goal Test: False
visiting (1,1), cost 2, points -1, Goal Test: False
visiting (2,1), cost 3, points -1, Goal Test: False
visiting (2,0), cost 4, points 9, Goal Test: False
visiting (3,1), cost 5, points 8, Goal Test: False
visiting (4,1), cost 6, points 13, Goal Test: False
visiting (4,0), cost 7, points 18, Goal Test: False
visiting (5,0), cost 8, points 23, Goal Test: False
visiting (4,2), cost 7, points 10, Goal Test: False
visiting (4,3), cost 8, points 12, Goal Test: False
visiting (5,2), cost 8, points 12, Goal Test: False
visiting (3,3), cost 9, points 9, Goal Test: False
visiting (3,4), cost 10, points 16, Goal Test: False
visiting (4,5), cost 11, points 23, Goal Test: False
visiting (5,5), cost 12, points 23, Goal Test: True
Minimum cost: 12, Maximum points: 23, Time Complexity: 0.0005042552947998047, Space Complexity: 15, Actual Memory used: 0.006193


### 5.	Comparitive Analysis

In [156]:
#Code Block : Print the Time & Space complexity of algorithm 1

print("Recursive Best First Search - A* (RBFS)")
print(f"Minimum cost: {min_cost_rbfs}, Maximum points: {max_points_rbfs}, Space complexity: {space_complexity_rbfs}, Time complexity: {time_complexity_rbfs} ms")

# The output of this function will display the time complexity (execution time)
# and the space complexity (number of nodes expanded) for the RBFS algorithm.

Recursive Best First Search - A* (RBFS)
Minimum cost: 12, Maximum points: 27, Space complexity: 16, Time complexity: 0.00019311904907226562 ms


In [157]:
#Code Block : Print the Time & Space complexity of algorithm 2

# The output of this function will display the time complexity (execution time)
# and the space complexity (number of nodes expanded) for the UCS algorithm.
print("Uniform Cost Search (UCS)")
print(f"Minimum cost: {min_cost_ucs}, Maximum points: {max_points_ucs}, Space complexity: {space_complexity_ucs}, Time complexity: {time_complexity_ucs} ms")

Uniform Cost Search (UCS)
Minimum cost: 12, Maximum points: 23, Space complexity: 15, Time complexity: 0.0005042552947998047 ms


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

### Comparison

Recursive Best First Search (RBFS) and Uniform Cost Search (UCS) show similar minimum costs, but UCS has a slightly lower maximum points value and a marginally higher space complexity. RBFS demonstrates faster time complexity (0.00019311904907226562 ms vs. 0.0005042552947998047 ms), but UCS might be preferred if space is more constrained.

Both algorithms will find the shortest path, but they operate differently. RBFS uses a heuristic to guide the search, while UCS explores all possible paths in order of their cost. The exact path found will depend on the specific grid and blocked paths. This path can be found using either RBFS or UCS, ensuring the shortest and safest route is taken.